In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the autoencoder neural network
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid(),  # To get the output in the range [0, 1]
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Hyperparameters
learning_rate = 0.001
batch_size = 64
epochs = 10

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the autoencoder, criterion, and optimizer
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    for batch in train_loader:
        inputs, _ = batch
        inputs = inputs.view(-1, 28 * 28)
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs, inputs)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'autoencoder.pth')


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

Epoch [1/10], Loss: 0.9240
Epoch [2/10], Loss: 0.9237
Epoch [3/10], Loss: 0.9229
Epoch [4/10], Loss: 0.9239
Epoch [5/10], Loss: 0.9225
Epoch [6/10], Loss: 0.9295
Epoch [7/10], Loss: 0.9280
Epoch [8/10], Loss: 0.9246
Epoch [9/10], Loss: 0.9268
Epoch [10/10], Loss: 0.9264


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 400),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(400, 20)  # Mean of the latent space
        self.fc_logvar = nn.Linear(400, 20)  # Log variance of the latent space
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(20, 400),
            nn.ReLU(),
            nn.Linear(400, 28 * 28),
            nn.Sigmoid(),
        )
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = self.fc_mu(h), self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Hyperparameters
learning_rate = 0.001
batch_size = 64
epochs = 10

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the VAE, criterion, and optimizer
model = VAE()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        inputs, _ = batch
        inputs = inputs.view(-1, 28 * 28)
        
        # Forward pass
        recon_batch, mu, logvar = model(inputs)
        
        # Compute the loss
        loss = loss_function(recon_batch, inputs, mu, logvar)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss / len(train_loader.dataset):.4f}')

# Save the trained model
torch.save(model.state_dict(), 'vae.pth')


Epoch [1/10], Loss: 147.8166
Epoch [2/10], Loss: 115.9736
Epoch [3/10], Loss: 111.4855
Epoch [4/10], Loss: 109.4452
Epoch [5/10], Loss: 108.1665
Epoch [6/10], Loss: 107.3684
Epoch [7/10], Loss: 106.7564
Epoch [8/10], Loss: 106.2049
Epoch [9/10], Loss: 105.8629
Epoch [10/10], Loss: 105.5453


In [3]:
def reparameterize(self, mu, logvar):
    std = torch.exp(0.5 * logvar)  # Calculate the standard deviation
    eps = torch.randn_like(std)  # Generate a random tensor with elements drawn from a standard normal distribution
    return mu + eps * std  # Reparameterize to obtain a sample from the latent distribution
